## RFM-Analysis

### Set-Up

In [3]:
# import libraries
import polars as pl
import os
from datetime import datetime, timedelta

import plotly.graph_objects as go
import plotly.express as px

In [4]:
import plotly.io as pio
pio.renderers.default = "vscode"

In [5]:
# define input path
input_path = "C:\\Users\\agste\\Angelos Work Projects\\RFM & Clustering Project\\Data_Lake\\cleaned_data.parquet"

# load cleaned dataset
print("Loading cleaned dataset...")
df = pl.read_parquet(input_path)
print(df.head())

Loading cleaned dataset...
shape: (5, 9)
┌───────────┬───────────┬────────────┬──────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ InvoiceNo ┆ StockCode ┆ Descriptio ┆ Quantity ┆ … ┆ UnitPrice ┆ CustomerI ┆ Country   ┆ Monetary │
│ ---       ┆ ---       ┆ n          ┆ ---      ┆   ┆ ---       ┆ D         ┆ ---       ┆ ---      │
│ i64       ┆ str       ┆ ---        ┆ i64      ┆   ┆ f64       ┆ ---       ┆ str       ┆ f64      │
│           ┆           ┆ str        ┆          ┆   ┆           ┆ i64       ┆           ┆          │
╞═══════════╪═══════════╪════════════╪══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ 567853    ┆ 22853     ┆ CAT BOWL   ┆ 2        ┆ … ┆ 3.25      ┆ 15484     ┆ United    ┆ 6.5      │
│           ┆           ┆ VINTAGE    ┆          ┆   ┆           ┆           ┆ Kingdom   ┆          │
│           ┆           ┆ CREAM      ┆          ┆   ┆           ┆           ┆           ┆          │
│ 550764    ┆ 22427     ┆ ENAMEL     ┆ 1        ┆ 

In [6]:
# define max date of the dataset (upper threshold)
max_date = df["InvoiceDate"].max()

# set reference date: last day of the previous month from max date 
ref_date = max_date.replace(day=1) - timedelta(days=1)

In [7]:
# define rfm path
rfm_path = "C:\\Users\\agste\\Angelos Work Projects\\RFM & Clustering Project\\Data_Lake\\rfm_segments.csv"
# load rfm segment table
rfm_segments = pl.read_csv(rfm_path)

### RFM Model

In [8]:
def rfm_model(
    df: pl.DataFrame,
    ref_date: pl.Date,
    segments_df: pl.DataFrame,
    output_columns: list = ["CustomerID", "Segment"],
) -> pl.DataFrame:
    """
    Calculate RFM scores and map them to customer segments.

    Parameters:
    - df (pl.DataFrame): The transaction data containing "CustomerID", "InvoiceDate", "InvoiceNo", and "Monetary".
    - ref_date (str): The reference date to calculate "Recency" (format: "YYYY-MM-DD").
    - segments_df (pl.DataFrame): A lookup table mapping RFM scores to segments.
    - output_columns (list, optional): The list of columns to return. Default: ["CustomerID", "Segment"], # ["Recency", "Frequency", "Monetary", "R", "F", "M", "RFM_Score"]

    Returns:
    - pl.DataFrame: The resulting dataset containing RFM metrics and segments.
    """

    # aggregate RFM metrics
    rfm_data = (
        df.filter(pl.col("InvoiceDate").le(ref_date))
        .group_by("CustomerID")
        .agg(
            (pl.lit(ref_date) - pl.col("InvoiceDate").max())
            .dt.total_days()
            .alias("Recency"),  # Days since last purchase
            pl.col("InvoiceNo")
            .n_unique()
            .alias("Frequency"),  # Number of unique purchases
            pl.col("Monetary").sum().alias("Monetary"),  # Total spend
        )
    )

    # RFM quantiles
    rfm_cols = ["Recency", "Frequency", "Monetary"]
    rfm_data = rfm_data.with_columns(
        [
            pl.col(column)
            .qcut(
                quantiles=4,
                labels=["1", "2", "3", "4"],
                left_closed=False,
                allow_duplicates=False,
            )
            .cast(pl.Int8)
            .alias(column[0])  # Rename to R, F, M
            for column in rfm_cols
        ]
    )

    # reverse R score for Recency (since lower Recency is better)
    rfm_data = rfm_data.with_columns([(5 - pl.col("R")).alias("R")])

    # create RFM Score column
    rfm_data = rfm_data.with_columns(
        pl.concat_str([pl.col("R"), pl.col("F"), pl.col("M")]).alias("RFM_Score")
    )

    # ensure RFM_Score in segments_df is string type
    segments_df = segments_df.cast({"RFM_Score": pl.Utf8})

    # join with RFM segments lookup table
    rfm_data = rfm_data.join(segments_df, on="RFM_Score", how="left", validate="m:1")

    # define fixed columns that must always appear first
    fixed_columns = ["CustomerID", "Segment"]

    # ensure additional columns are valid and exist in the dataset
    if output_columns:
        output_columns = [
            col
            for col in output_columns
            if col in rfm_data.columns and col not in fixed_columns
        ]
    else:
        output_columns = []

    # ensure final column order: CustomerID, Segment, followed by user-selected columns
    final_columns = fixed_columns + output_columns

    # Return only the requested columns
    return rfm_data.select(final_columns)

In [9]:
# run rfm_model & save output
rfm_output = rfm_model(
    df=df,
    ref_date=ref_date,
    segments_df=rfm_segments,
    output_columns=["Recency", "Frequency", "Monetary"],
).rename({"Segment": "RFM_Segment"})

### RFM Summary

In [10]:
# RFM Summary Table: group by RFM Segment and get total counts and avg values of the R, F, M metrics
rfm_summary = rfm_output.group_by("RFM_Segment").agg(
    pl.len().alias("Count"),
    pl.col("Recency").mean().round(2),
    pl.col("Frequency").mean().round(2),
    pl.col("Monetary").mean().round(2),
)
rfm_summary

RFM_Segment,Count,Recency,Frequency,Monetary
str,u32,f64,f64,f64
"""Loyal_Customers""",424,22.22,3.93,1287.41
"""At_Risk_Customers""",2,73.5,6.5,135.57
"""Lost Customers""",1519,181.48,1.35,320.31
"""Champions""",844,13.07,11.76,6531.11
"""Potential_Big_Spenders""",518,64.38,4.08,1439.21
"""Potential_Customers""",73,12.1,2.48,1205.26
"""Newcomers""",97,119.24,2.65,3765.04
"""Win_Back_Targets""",696,62.22,1.52,630.59
"""Fading_Customers""",120,31.96,2.98,353.91


### Summary Plots 

In [11]:
# Count vs. Monetary value per segment
fig1 = px.bar(
    rfm_summary,
    x="RFM_Segment",
    y="Monetary",
    color="RFM_Segment",
    title="Monetary Value per RFM Segment",
    labels={"Monetary": "Total Monetary Value", "RFM_Segment": "Customer Segment"},
    text="Count",
)
fig1.show()

In [12]:
# Recency vs. Frequency with size representing Monetary
fig2 = px.scatter(
    rfm_summary,
    x="Recency",
    y="Frequency",
    size="Monetary",
    color="RFM_Segment",
    title="Recency vs Frequency (Bubble Size Represents Monetary Value)",
    labels={"Recency": "Average Recency (Days)", "Frequency": "Purchase Frequency"},
    hover_name="RFM_Segment",
)
fig2.show()

In [13]:
# create new column - average basket size
rfm_summary = rfm_summary.with_columns(
    pl.col("Monetary")
    .truediv(pl.col("Frequency"))
    .round(2)
    .alias("Average_Basket_Value")
)

In [14]:
# Frequency to Monetary Ratio per segment
fig3 = px.bar(
    rfm_summary,
    x="RFM_Segment",
    y="Average_Basket_Value",
    color="RFM_Segment",
    title="Average Basket Value per RFM Segment",
    labels={
        "Average_Basket_Value": "Average_Basket_Value",
        "RFM_Segment": "Customer RFM Segment",
    },
    text="Count",
)
fig3.show()

### Save

In [15]:
# define output path
output_dir = "C:\\Users\\agste\\Angelos Work Projects\\RFM & Clustering Project\\Data_Lake"
os.makedirs(output_dir, exist_ok=True)  # Create directory if not exists
data_output_path = os.path.join(output_dir, "rfm_data.parquet")

In [16]:
# save cleaned data as Parquet
rfm_output.write_parquet(data_output_path)
print(f"Cleaned data saved to {data_output_path}")

Cleaned data saved to C:\Users\agste\Angelos Work Projects\RFM & Clustering Project\Data_Lake\rfm_data.parquet
